In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from random import shuffle
import cv2

In [2]:
dataset_folder = './dataset/'

face_path  = dataset_folder + 'face'
nface_path = dataset_folder + 'nonface'
testing_face_path = dataset_folder + 'testing/face'
testing_nface_path = dataset_folder + 'testing/nonface'
img_ext = ['.jpg', '.png', 'jpeg', '.pgm', '.bmp', 'JPEG']

height, width = 128, 128

In [3]:
def read_face_files(file_path=face_path):
    face_list = []
    
    for f in glob.glob(file_path + '/**', recursive=True):
        
        if f[-4:] in img_ext:
            face_list.append(f)
    
    return face_list

def read_nonface_files(file_path=nface_path):
    
    nonface_list = []
    
    for f in glob.glob(file_path + '/**', recursive=True):
        
        if f[-4:] in img_ext:
            nonface_list.append(f)
    
    return nonface_list

In [4]:
face_list = read_face_files()
nonface_list = read_nonface_files()

num_faces = len(face_list)
num_nonface = len(nonface_list)

face_label = [0] * num_faces
nonface_label = [1] * num_nonface

data_list = face_list + nonface_list
label_list = face_label + nonface_label

In [ ]:
def shuffle_list(list1, list2):
    
    combined = list(zip(list1, list2))
    shuffle(combined)

    list1, list2 = zip(*combined)

    return (list1, list2)

data_list, label_list = shuffle_list(data_list, label_list)

In [ ]:
def read_images(data_list=data_list, label_list=label_list):
    image_list = []
    one_hot_list = []
    for d in data_list:
        
        img = cv2.imread(d, 0)
        img = cv2.resize(img, (height,width))
        img = np.asarray(img, dtype='float32')/255
        img = np.reshape(img, (height, width, 1))
        image_list.append(img)
    
    for l in label_list:
        
        zeros = np.zeros((2))
        zeros[int(l)] = 1.0
        one_hot_list.append(zeros)
    
    labels = np.asarray(one_hot_list, dtype='float32')
    images = np.asarray(image_list, dtype='float32')
    
    return (images, labels)

images, labels = read_images()

In [ ]:
face_list = read_face_files(testing_face_path)
nonface_list = read_nonface_files(testing_nface_path)

num_faces = len(face_list)
num_nonface = len(nonface_list)

face_label = [0] * num_faces
nonface_label = [1] * num_nonface

data_list = face_list + nonface_list
label_list = face_label + nonface_label

data_list, label_list = shuffle_list(data_list, label_list)

test_images,test_labels = read_images(data_list, label_list)

## Convolutional Neural Network

In [ ]:
import tensorflow as tf
from tqdm import tqdm
from tabulate import tabulate #to print the output of our model in a tabular manner.

In [ ]:
batch_size           = 5
learning_rate        = 1e-4
epoch                = 20
num_labels           = 2
dropout_rate         = 0.5

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, height, width, 1), name='input')
Y = tf.placeholder(tf.float32, shape=(None, num_labels), name='target')
dropout_prob = tf.placeholder(tf.float32, name='dropout')


conv1 = tf.contrib.layers.conv2d(X, num_outputs=128, kernel_size=3, stride=1, padding='SAME',
                                 activation_fn=tf.nn.relu, scope='Conv_1')

conv1_pool = max_pool_2x2(conv1)

conv2 = tf.contrib.layers.conv2d(conv1_pool, num_outputs=256, kernel_size=3, stride=1, padding='SAME',
                                 activation_fn=tf.nn.relu, scope='Conv_2')
conv3 = tf.contrib.layers.conv2d(conv2, num_outputs=256, kernel_size=5, stride=1, padding='SAME',
                                 activation_fn=tf.nn.relu, scope='Conv_3')

conv2_pool = max_pool_2x2(conv3)

conv4 = tf.contrib.layers.conv2d(conv2_pool, num_outputs=512, kernel_size=5, stride=1, padding='SAME',
                                 activation_fn=tf.nn.relu, scope='Conv_4')

conv5 = tf.contrib.layers.conv2d(conv4, num_outputs=512, kernel_size=5, stride=1, padding='SAME',
                                 activation_fn=tf.nn.relu, scope='Conv_5')

conv5_pool = max_pool_2x2(conv5)

conv6 = tf.contrib.layers.conv2d(conv5_pool, num_outputs=512, kernel_size=3, stride=1, padding='SAME',
                                 activation_fn=tf.nn.relu, scope='Conv_6')

conv6_pool = max_pool_2x2(conv6)


W_fc1 = weight_variable([8 * 8 * 512, 512])
b_fc1 = bias_variable([512])

h_pool3_flat = tf.reshape(conv6_pool, [-1, 8*8*512])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

fc1_dropout = tf.nn.dropout(h_fc1, dropout_prob)

W_fc2 = weight_variable([512, num_labels])
b_fc2 = bias_variable([num_labels])

y_conv = tf.add(tf.matmul(fc1_dropout, W_fc2), b_fc2, name='output')


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_conv),name='loss')
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
#     print(shape)
#     print(len(shape))
    variable_parameters = 1
    for dim in shape:
#         print(dim)
        variable_parameters *= dim.value
#     print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

30905090


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver(tf.global_variables())
writer = tf.summary.FileWriter('./tensorboard')
writer.add_graph(sess.graph)

#to load a saved model
try:    
    saver.restore(sess, './cnn_ckpt/model.ckpt')
    print("Model is loaded!")

except Exception as e:
    print("Model is not loaded !")
    pass

total_images = images.shape[0] #total number of training images
total_test_images = test_images.shape[0] #total number of testing images

best_accuracy = 0

for epoch_iter in range(epoch):
    
    total_loss = 0
    total_accuracy = 0
    test_total_loss = 0
    test_total_accuracy = 0
    counter = 0
    test_counter= 0
    
    for index in tqdm(range(0, total_images, batch_size)):
        
        end_batch = index + batch_size
        
        if end_batch >= total_images : end_batch = None #to prevent the last index to go beyond the data size
            
        loss_, accuracy_, _ = sess.run([cross_entropy, accuracy, train_step],
                                                  feed_dict={X:images[index:end_batch], 
                                                             Y:labels[index:end_batch],
                                                             dropout_prob:dropout_rate})
        total_loss += loss_
        total_accuracy += accuracy_
        
        counter += 1
    
    for test_index in tqdm(range(0, total_test_images, batch_size)):
        
        end_batch_test = test_index + batch_size 
        
        if end_batch_test >= total_test_images : end_batch_test = None #to prevent the last index to go beyond the data size
        
        loss_, accuracy_ = sess.run([cross_entropy, accuracy],
                                                  feed_dict={X:test_images[test_index:end_batch_test], 
                                                             Y:test_labels[test_index:end_batch_test],
                                                             dropout_prob:1.0})
        
        test_total_loss += loss_
        test_total_accuracy += accuracy_
        test_counter += 1
        
    print("\t----Epoch %d---- "%(epoch_iter))
    print(tabulate([["Loss", total_loss],["Classification Accuracy", total_accuracy/counter]],
                       headers=["Training Loss/Accuracy", "Value"]))
    print("\n")
    print(tabulate([["Loss", test_total_loss],["Classification Accuracy", test_total_accuracy/test_counter]], 
                       headers=["Testing Loss/Accuracy", "Value"]))
    
    
    if test_total_accuracy/test_counter > best_accuracy :
        
        saver.save(sess, "./cnn_ckpt/model.ckpt")
        print("Model saved !")
        best_accuracy = test_total_accuracy/test_counter

INFO:tensorflow:Restoring parameters from ./cnn_ckpt/model.ckpt


  0%|          | 0/3880 [00:00<?, ?it/s]

Model is not loaded !


 61%|██████    | 2366/3880 [17:38<10:43,  2.35it/s]